<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Initial-data-preparation" data-toc-modified-id="Initial-data-preparation-1">Initial data preparation</a></span></li><li><span><a href="#Exploratory-data-analysis" data-toc-modified-id="Exploratory-data-analysis-2">Exploratory data analysis</a></span></li><li><span><a href="#Feature-importance" data-toc-modified-id="Feature-importance-3">Feature importance</a></span></li><li><span><a href="#One-hot-encoding" data-toc-modified-id="One-hot-encoding-4">One-hot encoding</a></span></li><li><span><a href="#Training-logistic-regression" data-toc-modified-id="Training-logistic-regression-5">Training logistic regression</a></span></li><li><span><a href="#Model-interpretation" data-toc-modified-id="Model-interpretation-6">Model interpretation</a></span></li><li><span><a href="#Using-the-model" data-toc-modified-id="Using-the-model-7">Using the model</a></span></li></ul></div>

Data from https://www.kaggle.com/blastchar/telco-customer-churn

In [1]:
import pandas as pd
import numpy as np

from IPython.display import display

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
churn = pd.read_excel(r'C:\Users\wolff\Desktop\DataJam\Customers Churn Rate.xlsx')

In [3]:
len(churn)

7043

## Initial data preparation

In [4]:
churn.head()

,UserID,Gender,SeniorCitizen,Status,Dependents,LoyaltyPeriod,PhoneService,MultipleLines,InternetService,Security,...,StreamingTV,StreamingMovies,ContractType,EmailBilling,PaymentMethod,MonthlyCharges,TotalCharges,numAdminTickets,numTechTickets,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,0,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,One year,No,Mailed check,56.95,1889.5,0,0,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,0,0,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,3,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,0,0,Yes


In [5]:
churn.head().T

,0,1,2,3,4
UserID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
Gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Status,Yes,No,No,No,No
Dependents,No,No,No,No,No
LoyaltyPeriod,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
Security,No,Yes,Yes,Yes,No


In [6]:
churn.dtypes

UserID              object
Gender              object
SeniorCitizen        int64
Status              object
Dependents          object
LoyaltyPeriod        int64
PhoneService        object
MultipleLines       object
InternetService     object
Security            object
Backup              object
Protection          object
Support             object
StreamingTV         object
StreamingMovies     object
ContractType        object
EmailBilling        object
PaymentMethod       object
MonthlyCharges     float64
TotalCharges        object
numAdminTickets      int64
numTechTickets       int64
Churn               object
dtype: object

In [7]:
churn['TotalCharges'] = pd.to_numeric(churn['TotalCharges'], errors='coerce')
churn['TotalCharges'] = churn['TotalCharges'].fillna(0)

In [8]:
churn.columns = churn.columns.str.lower().str.replace(' ', '_')

string_columns = list(churn.dtypes[churn.dtypes == 'object'].index)

for col in string_columns:
    churn[col] = churn[col].str.lower().str.replace(' ', '_')

In [9]:
churn.churn = (churn.churn == 'yes').astype(int)

In [10]:
churn.head().T

,0,1,2,3,4
userid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
gender,female,male,male,male,female
seniorcitizen,0,0,0,0,0
status,yes,no,no,no,no
dependents,no,no,no,no,no
loyaltyperiod,1,34,2,45,2
phoneservice,no,yes,yes,no,yes
multiplelines,no_phone_service,no,no,no_phone_service,no
internetservice,dsl,dsl,dsl,dsl,fiber_optic
security,no,yes,yes,yes,no


In [11]:
churn_train_full, churn_test = train_test_split(churn, test_size=0.2, random_state=1)

In [12]:
churn_train, churn_val = train_test_split(churn_train_full, test_size=0.33, random_state=11)

In [13]:
y_train = churn_train.churn.values
y_val = churn_val.churn.values

In [14]:
del churn_train['churn']
del churn_val['churn']

## Exploratory data analysis

In [15]:
churn_train_full.isnull().sum()

userid             0
gender             0
seniorcitizen      0
status             0
dependents         0
loyaltyperiod      0
phoneservice       0
multiplelines      0
internetservice    0
security           0
backup             0
protection         0
support            0
streamingtv        0
streamingmovies    0
contracttype       0
emailbilling       0
paymentmethod      0
monthlycharges     0
totalcharges       0
numadmintickets    0
numtechtickets     0
churn              0
dtype: int64

In [16]:
churn_train_full.churn.value_counts()

0    4113
1    1521
Name: churn, dtype: int64

In [17]:
global_mean = churn_train_full.churn.mean()
round(global_mean, 3)

0.27

In [18]:
categorical = ['gender', 'seniorcitizen', 'status', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'security', 'backup', 'protection',
               'support', 'streamingtv', 'streamingmovies',
               'contracttype', 'emailbilling', 'paymentmethod']
numerical = ['loyaltyperiod', 'monthlycharges', 'totalcharges','numadmintickets','numtechtickets']

In [19]:
churn_train_full.columns

Index(['userid', 'gender', 'seniorcitizen', 'status', 'dependents',
       'loyaltyperiod', 'phoneservice', 'multiplelines', 'internetservice',
       'security', 'backup', 'protection', 'support', 'streamingtv',
       'streamingmovies', 'contracttype', 'emailbilling', 'paymentmethod',
       'monthlycharges', 'totalcharges', 'numadmintickets', 'numtechtickets',
       'churn'],
      dtype='object')

In [20]:
churn_train_full[categorical].nunique()

gender             2
seniorcitizen      2
status             2
dependents         2
phoneservice       2
multiplelines      3
internetservice    3
security           3
backup             3
protection         3
support            3
streamingtv        3
streamingmovies    3
contracttype       3
emailbilling       2
paymentmethod      4
dtype: int64

## Feature importance

In [21]:
female_mean = churn_train_full[churn_train_full.gender == 'female'].churn.mean()
print('gender == female:', round(female_mean, 3))

male_mean = churn_train_full[churn_train_full.gender == 'male'].churn.mean()
print('gender == male:  ', round(male_mean, 3))

gender == female: 0.277
gender == male:   0.263


In [22]:
female_mean / global_mean

1.0253955354648652

In [23]:
male_mean / global_mean

0.9749802969838747

In [24]:
churn_group = churn_train_full.groupby(by='gender').churn.agg(['mean'])
churn_group['diff'] = churn_group['mean'] - global_mean
churn_group['risk'] = churn_group['mean'] / global_mean
churn_group

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


In [25]:
global_mean = churn_train_full.churn.mean()
global_mean

0.26996805111821087

In [26]:
for col in categorical:
    churn_group = churn_train_full.groupby(by=col).churn.agg(['mean'])
    churn_group['diff'] = churn_group['mean'] - global_mean
    churn_group['risk'] = churn_group['mean'] / global_mean
    display(churn_group)

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,risk
status,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,risk
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,risk
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


...

,mean,diff,risk
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,risk
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,risk
contracttype,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,risk
emailbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121


In [27]:
def calculate_mi(series):
    return mutual_info_score(series, churn_train_full.churn)

churn_mi = churn_train_full[categorical].apply(calculate_mi)
churn_mi = churn_mi.sort_values(ascending=False).to_frame(name='MI')


display(churn_mi.head())
display(churn_mi.tail())

,MI
contracttype,0.098320
security,0.063085
support,0.061032
internetservice,0.055868
backup,0.046923


,MI
status,0.009968
seniorcitizen,0.009410
multiplelines,0.000857
phoneservice,0.000229
gender,0.000117


In [28]:
churn_train_full[numerical].corrwith(churn_train_full.churn).to_frame('correlation')

,correlation
loyaltyperiod,-0.351885
monthlycharges,0.196805
totalcharges,-0.196353
numadmintickets,-0.015086
numtechtickets,0.359577


In [29]:
churn_train_full.groupby(by='churn')[numerical].mean()

,loyaltyperiod,monthlycharges,totalcharges,numadmintickets,numtechtickets
churn,,,,,
0,37.531972,61.176477,2548.021627,0.526623,0.152443
1,18.070348,74.521203,1545.689415,0.483235,1.179487


## One-hot encoding

In [30]:
train_dict = churn_train[categorical + numerical].to_dict(orient='records')

In [31]:
train_dict[0]

{'gender': 'male',
 'seniorcitizen': 0,
 'status': 'yes',
 'dependents': 'no',
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'dsl',
 'security': 'yes',
 'backup': 'yes',
 'protection': 'yes',
 'support': 'yes',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contracttype': 'two_year',
 'emailbilling': 'yes',
 'paymentmethod': 'bank_transfer_(automatic)',
 'loyaltyperiod': 71,
 'monthlycharges': 86.1,
 'totalcharges': 6045.9,
 'numadmintickets': 0,
 'numtechtickets': 0}

In [32]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [33]:
X_train = dv.transform(train_dict)

In [34]:
X_train.shape

(3774, 47)

In [35]:
dv.get_feature_names_out()

array(['backup=no', 'backup=no_internet_service', 'backup=yes',
       'contracttype=month-to-month', 'contracttype=one_year',
       'contracttype=two_year', 'dependents=no', 'dependents=yes',
       'emailbilling=no', 'emailbilling=yes', 'gender=female',
       'gender=male', 'internetservice=dsl',
       'internetservice=fiber_optic', 'internetservice=no',
       'loyaltyperiod', 'monthlycharges', 'multiplelines=no',
       'multiplelines=no_phone_service', 'multiplelines=yes',
       'numadmintickets', 'numtechtickets',
       'paymentmethod=bank_transfer_(automatic)',
       'paymentmethod=credit_card_(automatic)',
       'paymentmethod=electronic_check', 'paymentmethod=mailed_check',
       'phoneservice=no', 'phoneservice=yes', 'protection=no',
       'protection=no_internet_service', 'protection=yes', 'security=no',
       'security=no_internet_service', 'security=yes', 'seniorcitizen',
       'status=no', 'status=yes', 'streamingmovies=no',
       'streamingmovies=no_internet_

## Training logistic regression

In [36]:
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [37]:
val_dict = churn_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [38]:
model.predict_proba(X_val)

array([[0.89035577, 0.10964423],
       [0.86593066, 0.13406934],
       [0.97058833, 0.02941167],
       ...,
       [0.9512968 , 0.0487032 ],
       [0.38103089, 0.61896911],
       [0.97019529, 0.02980471]])

In [39]:
y_pred = model.predict_proba(X_val)[:, 1]

In [40]:
y_pred

array([0.10964423, 0.13406934, 0.02941167, ..., 0.0487032 , 0.61896911,
       0.02980471])

In [41]:
churn_s = y_pred > 0.5

In [42]:
(y_val == churn_s).mean()

0.8473118279569892

## Model interpretation

In [43]:
model.intercept_[0]

-0.13636096912420012

In [44]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))

{'backup=no': 0.257,
 'backup=no_internet_service': -0.114,
 'backup=yes': -0.28,
 'contracttype=month-to-month': 0.577,
 'contracttype=one_year': -0.019,
 'contracttype=two_year': -0.694,
 'dependents=no': -0.109,
 'dependents=yes': -0.028,
 'emailbilling=no': -0.227,
 'emailbilling=yes': 0.091,
 'gender=female': -0.005,
 'gender=male': -0.131,
 'internetservice=dsl': -0.441,
 'internetservice=fiber_optic': 0.418,
 'internetservice=no': -0.114,
 'loyaltyperiod': -0.08,
 'monthlycharges': 0.003,
 'multiplelines=no': -0.216,
 'multiplelines=no_phone_service': 0.066,
 'multiplelines=yes': 0.014,
 'numadmintickets': 0.013,
 'numtechtickets': 1.372,
 'paymentmethod=bank_transfer_(automatic)': 0.047,
 'paymentmethod=credit_card_(automatic)': -0.158,
 'paymentmethod=electronic_check': 0.107,
 'paymentmethod=mailed_check': -0.134,
 'phoneservice=no': 0.066,
 'phoneservice=yes': -0.202,
 'protection=no': 0.179,
 'protection=no_internet_service': -0.114,
 'protection=yes': -0.201,
 'security=no

In [45]:
subset = ['contracttype', 'loyaltyperiod', 'totalcharges']
train_dict_small = churn_train[subset].to_dict(orient='records')
dv_small = DictVectorizer(sparse=False)
dv_small.fit(train_dict_small)

X_small_train = dv_small.transform(train_dict_small)

dv_small.get_feature_names_out()

array(['contracttype=month-to-month', 'contracttype=one_year',
       'contracttype=two_year', 'loyaltyperiod', 'totalcharges'],
      dtype=object)

In [46]:
model_small = LogisticRegression(solver='liblinear', random_state=1)
model_small.fit(X_small_train, y_train)

LogisticRegression(random_state=1, solver='liblinear')

In [47]:
model_small.intercept_[0]

-0.6384442006590022

In [48]:
dict(zip(dv_small.get_feature_names_out(), model_small.coef_[0].round(3)))

{'contracttype=month-to-month': 0.909,
 'contracttype=one_year': -0.145,
 'contracttype=two_year': -1.403,
 'loyaltyperiod': -0.097,
 'totalcharges': 0.001}

In [49]:
val_dict_small = churn_val[subset].to_dict(orient='records')
X_small_val = dv_small.transform(val_dict_small)

In [50]:
y_pred_small = model_small.predict_proba(X_small_val)[:, 1]

## Using the model

In [51]:
ch = churn.query('userid == "8879-zkjof"')
ch.numtechtickets

3381    0
Name: numtechtickets, dtype: int64

In [52]:
customer = {
    'userid': '8879-zkjof',
    'gender': 'female',
    'seniorcitizen': 0,
    'status': 'no',
    'dependents': 'no',
    'loyaltyperiod': 41,
    'phoneservice': 'yes',
    'multiplelines': 'no',
    'internetservice': 'dsl',
    'security': 'yes',
    'backup': 'no',
    'protection': 'yes',
    'support': 'yes',
    'streamingtv': 'yes',
    'streamingmovies': 'yes',
    'contracttype': 'one_year',
    'emailbilling': 'yes',
    'paymentmethod': 'bank_transfer_(automatic)',
    'monthlycharges': 79.85,
    'totalcharges': 3320.75,
    'numadmintickets': 3,
    'numtechtickets': 0
}

In [53]:
X_test = dv.transform([customer])
model.predict_proba(X_test)[0, 1]

0.00895744689928296

In [60]:
churn.numtechtickets.min()
churn.numtechtickets.max()

9

In [55]:
print(list(X_test[0]))

[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 41.0, 79.85, 1.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 3320.75]


In [56]:
customer = {
    'gender': 'female',
    'seniorcitizen': 1,
    'status': 'no',
    'dependents': 'no',
    'phoneservice': 'yes',
    'multiplelines': 'yes',
    'internetservice': 'fiber_optic',
    'security': 'no',
    'backup': 'no',
    'protection': 'no',
    'techsupport': 'no',
    'streamingtv': 'yes',
    'streamingmovies': 'no',
    'contracttype': 'month-to-month',
    'emailbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'loyaltyperiod': 1,
    'monthlycharges': 85.7,
    'totalcharges': 85.7
}

In [57]:
X_test = dv.transform([customer])
model.predict_proba(X_test)[0, 1]

0.8942446805759428

In [58]:
pip install joblib